In [1]:
#READ_CSV, READ_TEXT, Import Pandas, os

import pandas as pd
import os

def read_csv(path):
    df = pd.read_csv(path, encoding='unicode_escape')
    df['PersonID'] = os.path.split(path)[1].split('_')[0][6:]
    df['ImageID'] = os.path.split(path)[1].split('.')[0][6:]
    return df

#只有 Text 跟 Caption 會用
def read_text(path, colName):
    df = pd.read_csv(path, sep='\t', encoding='unicode_escape', names=[colName])
    filename = os.path.split(path)[1]
    if 'text' in filename:
        df['TextID'] = filename[6:-4]
    else:
        df['ImageID'] = os.path.split(path)[1].split('.')[0][6:-7]
    return df


In [2]:
#Reading all file

DataPath = 'MC2-Image-Data/'

#Full DataPaths
CsvList = []
ImageList = []
CapList = []
TextList = []

ImagesNames = []

for r, d, files in os.walk(DataPath):

    if 'TrainingImages' in r:
            continue

    for _file in files:

        if _file.endswith('.csv'):
            CsvList.append(r + '/' + _file)
        elif _file.endswith('.jpg'):
            ImageList.append(r + '/' +_file)
            ImagesNames.append(_file)
        elif 'caption.txt' in _file:
            CapList.append(r + '/' +_file)
        elif 'text' in _file:
            TextList.append(r + '/' +_file)

In [3]:
# Make Caption/Text/Csv DataFrame

tmp = []
for path in CapList:
    tmp.append(read_text(path, 'Caption'))

cap_df = pd.concat(tmp, ignore_index=True)

tmp = []
for path in TextList:
    tmp.append(read_text(path, 'Text'))

text_df = pd.concat(tmp, ignore_index=True)

tmp = []
for path in CsvList:
    tmp.append(read_csv(path))

csv_df = pd.concat(tmp, ignore_index=True)

In [4]:
# Sort by Simply ImageID
csv_df['Sort'] = csv_df['ImageID'].apply(lambda x : int(x.replace('_', ""))) 
csv_df = csv_df.sort_values(by='Sort', ignore_index=True)
csv_df = csv_df.drop(columns = ['Sort'])



In [5]:
# Save Files
csv_df.to_json('Fine_Data', orient='records')
